# HW2: Regularized least squares, a.k.a. "Ridge regression"

## Note on the autograder:


For this homework, you will have access to Numpy, Matplotlib, Seaborn, and Plotly. If you use a different import, it probably will not work!

## Follow best practices in your code

While we won't grade your homeworks based on these subjective criteria, you should start getting practice following these guidelines as (a) we will be checking for these when we grade your project code and (b) these good programming habits will serve you very well in the future.

1. All functions should include informative docstrings.
2. You should include type hints for both the inputs and outputs of each new function you define.
3. Include short and informative inline comments throughout your code.

You will often find your solutions easier to write if you implement helper functions in addition to the required autograded functions in the template `.py` files. Feel free to add these to the same modules alongside the autograded functions.

Finally, **do not modify arguments passed into the autograded functions**. Such functions that mutate their inputs have what are called *side effects*: these symptoms can include extremely subtle bugs in other regions of your code and autograder crashes!

## Submission:

For this homework, you will submit your completed copy of this `.ipynb` notebook as well as `ridge.py`. In particular, see the Application section in this notebook: **you should fill out all of the empty cells to receive full credit**.



## Summary

Many real-world problems can be written as a least squares problem (for example, linear regression):
$$
\min_{x\in\mathbb{R}^d} \; \|Ax-y\|_2^2,
$$
(Here, $\|\cdot\|_2$ denotes the Euclidean norm, e.g., for a vector $z\in\mathbb{R}^m$, $\|z\|_2^2 = \sum_{i=1}^m z_i^2$.)

In the equation above, the matrix `A` has shape `(m, d)` and the vector `y` has shape `(m,)` (in math, we would usually say it's $m \times 1$, but in `numpy`, the convention is generally to flatten the array to only have one axis).

Least squares can be numerically unstable, especially when `A` is ill-conditioned. Adding a small penalty on the size of the coefficients improves numerical stability and can reduce overfitting by discouraging overly large weights. This is formalized by the regularized least squares (ridge regression) problem:
$$
\min_{x\in\mathbb{R}^d} \; \|Ax-y\|_2^2 + \lambda\,\|x\|_2^2,\quad \lambda>0.
$$
You can read more here: https://en.wikipedia.org/wiki/Ridge_regression.

In this assignment, you will write a solver for the regularized least squares problem.

Like standard least squares, the regularized least squares problem has a closed-form solution, and we can also use gradient descent (GD) or stochastic gradient descent (SGD) instead.

## Mathematical background

**Explicit formula:** The solution to the regularized least squares problem is
$$
 x = (A^\top A + \lambda I_d)^{-1}A^\top y,
$$
where `I_d` is the identity matrix of shape `(d, d)`. In other words, the optimal solution comes from the normal equation
$$
 (A^\top A + \lambda I_d) x = A^\top y.
$$

**GD:** To implement GD, we use the gradient of the objective function
$$
 f(x) = \|Ax-y\|_2^2 + \lambda \|x\|_2^2,\quad \nabla f(x) = 2(A^\top A + \lambda I_d) x - 2A^\top y.
$$

**SGD:** Computing the gradient using all samples can be expensive. To speed this up, use a minibatch of `b` samples to approximate the gradient:
$$
 \nabla f(x) \approx 2(A_b ^\top A_b + \lambda I_d) x - 2A_b^\top y_b,
$$
where `A_b` is obtained by randomly selecting `b` rows from `A` (keeping all columns), and `y_b` contains the corresponding entries from `y`.

Note: gradient descent (GD) is exactly the special case of SGD with `batch_size = m`. When `b = m`, we have $A_b = A$ and $y_b = y$, so the minibatch gradient matches the full gradient and the two methods are identical.

## Coding instructions:

Implement a solver for the regularized least squares problem in `ridge.py` (your renamed template file) by creating a class `LinearSolver` with the following behavior and method signatures. We have provided a template to get you started. Your implementation will be graded on these exact behaviors, defaults, and error messages, using the test cases listed farther below.

- `__init__(A, y)`
  - Inputs: `A` (2D numpy array with shape `(m, d)`), `y` (1D numpy array with shape `(m,)`).
  - Behavior:
    - If `A` is not 2D, raise `ValueError: Initialization failed because the A matrix is not a 2D numpy array.`
    - If `y` is not 1D, print `Warning: vector is not a 1D numpy array. Reshaped to 1D numpy array automatically.` and reshape to 1D automatically.
    - If `A.shape[0] != y.shape[0]`, raise `ValueError: Mismatched number of rows. Initialization failed.`

- `rls(reg)` - closed form solution of regularized least squares
  - `reg` (float | None): regularization strength ($\lambda$). Defaults to `1.0`.
  - Cases and required errors:
    - `reg < 0`: raise `ValueError: Regularization parameter should be nonnegative or None.`
    - `reg == 0` or `reg is None`: solve the unregularized least-squares problem using `np.linalg.solve`.
    - `reg > 0`: solve the ridge regression problem using `np.linalg.solve`.
  - Returns: coefficient vector `x` as a `numpy` array of shape `(d,)`.

- `sgd(reg, max_iter, batch_size, step_size)` - GD or SGD solution of regularized least squares
  - `reg` (nonnegative float): ridge penalty strength ($\lambda$). If `reg < 0`, raise `ValueError: Regularization parameter should be nonnegative or None.`
  - `max_iter` (int): maximum number of gradient descent steps. Default `100`.
  - `batch_size` (int): minibatch size (rows sampled without replacement each update). Default `2`. When `batch_size = m`, this is full‑batch GD.
  - `step_size` (float): learning rate ($\eta$). Default `0.01`.
  - Initialization: start from the zero vector `x = np.zeros(d)`.
  - Each step, perform the update with minibatch `(A_b, y_b)`: $$x \leftarrow x - \eta\,\big(2A_b^\top(A_b x - y_b) + 2\lambda x\big)$$
  - Returns: coefficient vector `x` as a `numpy` array of shape `(d,)`.

Docstrings should clearly explain the shapes of numpy arrays used by your class and methods.


In [2]:
from ridge import LinearSolver
import numpy as np

In [4]:
A = np.random.normal(loc=0, scale=1, size=(10, 20))
y = np.random.normal(loc=0, scale=1, size=(10,))
reg = LinearSolver(A, y)
reg.rls(reg=-1)

ValueError: Initialization failed because the A matrix is not a 2D numpy array.

## Tests

Your code will be graded based on the following test cases. Many of these tests have to do with validating the inputs, as it's really easy to make a subtle error such as passing the wrong size matrices for $A$ or $y$. (They should be easy to code up, though, since you don't actually need to run the regression for invalid inputs!)

#### Matrix shape with wrong number of axes is not accepted

If `A` is not a 2D numpy array, your constructor should raise `ValueError: Initialization failed because the A matrix is not a 2D numpy array.`

In [4]:
np.random.seed(0)
A = np.random.normal(loc=0, scale=1, size=(10, 20, 10))
y = np.random.normal(loc=0, scale=1, size=(40,))
reg = LinearSolver(A, y)

ValueError: Initialization failed because the A matrix is not a 2D numpy array.

#### Vector reshaping warning

If `y` is not 1D, print the warning and reshape `y` to a 1D vector automatically.

In [5]:
A = np.random.normal(loc=0, scale=1, size=(10, 20))
y = np.random.normal(loc=0, scale=1, size=(10, 1))
reg = LinearSolver(A, y)

#### Initialization error: mismatched rows

If the number of rows in `A` and the length of `y` do not match, raise `ValueError: "Mismatched number of rows. Initialization failed.`"

In [7]:
A = np.random.normal(loc=0, scale=1, size=(10, 20))
y = np.random.normal(loc=0, scale=1, size=(30, 2))
reg = LinearSolver(A, y)

ValueError: Mismatched number of rows. Initialization failed.

#### RLS error: negative regularization

If `reg < 0`, `rls` should raise a `ValueError` with the specified message.

In [4]:
A = np.random.normal(loc=0, scale=1, size=(10, 20))
y = np.random.normal(loc=0, scale=1, size=(10,))
reg = LinearSolver(A, y)
reg.rls(reg=-1)

ValueError: Regularization parameter should be nonnegative or None.

In [8]:
A = np.random.normal(loc=0, scale=1, size=(10, 20))
y = np.random.normal(loc=0, scale=1, size=(10,))
reg = LinearSolver(A, y)
reg.rls(reg=-1)

ValueError: Regularization parameter should be nonnegative or None.

#### RLS with default parameters

Calling `rls()` with no arguments should default to lambda=1 and return the corresponding ridge solution.

In [4]:
A = np.array([[1, 0], [0, 0]])
y = np.array([1, 1])
reg = LinearSolver(A, y)
reg.rls()

array([0.5, 0. ])

In [9]:
A = np.array([[1, 0], [0, 0]])
y = np.array([1, 1])
reg = LinearSolver(A, y)
reg.rls()

array([0.5, 0. ])

#### RLS without regularization

Calling `rls` with lambda=0 or lambda=None should return the unregularized least squares solution.

In [4]:
A = np.array([[1, 0], [0, 1], [1, 1]])
y = np.array([1, 1, 2])
reg = LinearSolver(A, y)
reg.rls(reg=None)

array([1., 1.])

In [10]:
A = np.array([[1, 0], [0, 1], [1, 1]])
y = np.array([1, 1, 2])
reg = LinearSolver(A, y)
reg.rls(reg=None)

array([1., 1.])

In [6]:
A = np.array([[1, 0], [0, 1], [1, 1]])
y = np.array([1, 1, 3])
reg = LinearSolver(A, y)
reg.rls(reg=0)

array([1.33333333, 1.33333333])

In [11]:
A = np.array([[1, 0], [0, 1], [1, 1]])
y = np.array([1, 1, 3])
reg = LinearSolver(A, y)
reg.rls(reg=0)

array([1.33333333, 1.33333333])

#### SGD error: negative regularization

If `reg < 0`, `sgd` should raise `ValueError: Regularization parameter should be nonnegative or None.`

In [4]:
A = np.random.normal(loc=0, scale=1, size=(10, 20))
y = np.random.normal(loc=0, scale=1, size=(10,))
reg = LinearSolver(A, y)
reg.sgd(reg=-1)

ValueError: Regularization parameter should be nonnegative or None.

In [ ]:
A = np.random.normal(loc=0, scale=1, size=(10, 20))
y = np.random.normal(loc=0, scale=1, size=(10,))
reg = LinearSolver(A, y)
reg.sgd(reg=-1)

ValueError: Regularization parameter should be nonnegative or None.

You may have different outputs for the next two test examples since SGD randomly 
selects training samples. For grading, your SGD solution is compared to a reference solution to check that the deviation is not statistically significant. In addition, the correct functioning of the SGD method will be relevant to the Application section below.

#### SGD (no regularization)



In [22]:
A = np.array([[1, 0], [0, 0], [0, 0]])
y = np.array([1, 1, 1])
reg = LinearSolver(A, y)
reg.sgd(reg=0)

array([0., 2.])

In [13]:
A = np.array([[1, 0], [0, 0], [0, 0]])
y = np.array([1, 1, 1])
reg = LinearSolver(A, y)
reg.sgd(reg=0)

array([0.69637434, 0.        ])

#### SGD (with regularization)

In [16]:
A = np.array([[1, 0], [0, 0], [0, 0]])
y = np.array([1, 1, 1])
reg = LinearSolver(A, y)
reg.sgd(reg=1, max_iter=100, batch_size=3, step_size=0.1)

array([0., 0.])

In [14]:
A = np.array([[1, 0], [0, 0], [0, 0]])
y = np.array([1, 1, 1])
reg = LinearSolver(A, y)
reg.sgd(reg=1, max_iter=100, batch_size=3, step_size=0.1)

array([0.5, 0. ])

## Application

Regularization is a crucial technique in machine learning. In this part, you will use your solver to compare least squares (call `rls` with lambda=0) and ridge regression (call `rls` with lambda>0), and then repeat with `sgd`.

The following code generates training and test data.

In [118]:
np.random.seed(19)

m = 100                                               # number of training samples
d = 100                                               # number of features
A = np.random.normal(0, 1, size=(m, d))               # training data matrix
x_true = np.random.uniform(size=(d,))                 # true coefficient vector
y = A @ x_true + np.random.normal(0, 0.5, size=(m,))  # training labels

m_test = 500                                          # number of test samples 
A_test = np.random.normal(0, 1, size=(m_test, d))     # test data matrix
y_test = A_test @ x_true                              # test labels

#### Least squares solution (closed form)
Find the least squares solution without regularization using training data $A$ and $y$ by calling `LinearSolver.rls` with lambda=0 to get $x_{LS}$.

Report two quantities:
1. The norm $\|x_{true} - x_{LS}\|$.
2. The mean squared error on the test set. Compute $y_{pred} = A_{test}\, x_{LS}$, then compute $MSE = \frac{1}{n}\sum_i (y_{test,i} - y_{pred,i})^2$

In [57]:
reg = LinearSolver(A, y)
x_LS = reg.rls(0)
print("norm:", np.linalg.norm(x_true - x_LS))
y_pred = A_test@x_LS
MSE = np.sum(np.square(y_test - y_pred))
print("Mean Squared Error:", MSE)

norm: 75.86377961234408
Mean Squared Error: 2907214.4792334177


#### Ridge regression (closed form)
Find a regularized solution using training data $A$ and $y$ by calling `LinearSolver.rls` with lambda>0 to get $x_{RLS}$.

Report two quantities, analogous to the least squares case:
1. $\|x_{true} - x_{RLS}\|$.
2. Test-set MSE for $x_{RLS}$.

Repeat this experiment for 3 values of $\lambda$ (e.g., $0.1, 1.0, 10.0$) and briefly state which you selected and why (based on test-set MSE).

In [76]:
reg = LinearSolver(A, y)
x_RLS = reg.rls(1.0)
print("norm:", np.linalg.norm(x_true - x_RLS))
y_pred = A_test@x_RLS
MSE = np.sum(np.square(y_test - y_pred))
print("Mean Squared Error:", MSE)

norm: 1.8621738686559521
Mean Squared Error: 1573.4598025855078


I selected $\lambda = 1.0$ since it resulted in the lowest MSE of about $1573$ whereas both $\lambda = 0.1$ and $\lambda = 10$ gave a MSE of over $2000$

#### Repeat with SGD

Using `LinearSolver.sgd`, compute:
- An unregularized solution (use lambda=0); report the norm $\|x_{true} - x_{\text{SGD, ls}}\|$ and the test-set MSE.
- A regularized solution (use your best lambda); report the norm $\|x_{true} - x_{\text{SGD, ridge}}\|$ and the test-set MSE.

Notes:
- You may need to try different max_iter, batch_size, and step_size to get good performance. Briefly state your chosen values.

In [124]:
# Unregularized solution
reg = LinearSolver(A, y)
x_SGDls = reg.sgd(reg=0, step_size=1e-4)
print("norm:", np.linalg.norm(x_true - x_SGDls))
y_pred = A_test@x_SGDls
MSE = np.sum(np.square(y_test - y_pred))
print("Mean Squared Error:", MSE)

norm: 5.503951433592289
Mean Squared Error: 14413.9611978744


In [126]:
# Regularized Solution
reg = LinearSolver(A, y)
x_SGDridge = reg.sgd(reg=1, step_size=1e-4)
print("norm:", np.linalg.norm(x_true - x_SGDridge))
y_pred = A_test@x_SGDridge
MSE = np.sum(np.square(y_test - y_pred))
print("Mean Squared Error:", MSE)

norm: 5.480878220661539
Mean Squared Error: 14074.954405999011


#### Does your experiment support that regularization is helpful in improving the model's performance? (Write answer below.)